---
# THIS IS THE REAL EVAL

In [ ]:
!pip3 install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
!pip3 install --quiet nibabel

In [1]:
import torch
import nibabel as nib
import os
import numpy as np

In [33]:
def eval(inputs, targets, smooth=0.001):
    # Smooth prevents miss matches from being too significant but keeps the child of fraction above zero.
    # from torch.nn import functional as F
    # inputs = F.sigmoid(inputs) # uncomment if inputs is not between 0-1

    inputs = inputs.flatten()
    targets = targets.flatten()

    intersection = (inputs * targets).sum()
    dice = (2.*intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

    # return 1 - dice
    # return -torch.log(dice)
    return dice

def ensemble(pred_2d, pred_3d):
    from torch.nn import functional as F
    # INPUTS ARE PROBABILITIES MATRICES
    fn_class =  lambda x: 1.0 * (x > 0.5) # network output image를 binary class로 분류해주는 class function정의
    # compare tensors and return bigger num
    # compare = lambda x, y : 0 if x > y else 1
    
    # pred_2d = F.softmax(pred_2d)
    # pred_3d = F.softmax(pred_3d)

    # twoD = compare(pred_2d[0].transpose(1,2,0), pred_2d[1].transpose(1,2,0))
    # threeD = compare(pred_3d[0].transpose(1,2,0), pred_3d[1].transpose(1,2,0))

    pred_2d_flatten = pred_2d.flatten()
    pred_3d_flatten = pred_3d.flatten()

    new_pred = (pred_2d_flatten * pred_3d_flatten)/2
    new_label = fn_class(new_pred)

    return new_label

def init_imgs(path): # PACK UP PATHS
    pred2d_list = os.listdir(os.path.join(path, "pred2d"))
    pred3d_list = os.listdir(os.path.join(path, "pred3d"))
    gt_list = os.listdir(os.path.join(path, "b4_raw"))
    
    temp_list = []
    for thing in pred2d_list:
        if ".nii.gz" in thing:
            mpath = os.path.join(path, "pred2d")
            temp_list.append(os.path.join(mpath, thing))
    pred2d_list = temp_list
    
    temp_list = []
    for thing in pred3d_list:
        if ".nii.gz" in thing:
            mpath = os.path.join(path, "pred3d")
            temp_list.append(os.path.join(mpath, thing))
    pred3d_list = temp_list

    pnum_list = [os.path.basename(file).split(".nii.gz")[0] for file in pred3d_list]

    temp_list = []
    for p in gt_list:
        if p in pnum_list:
            seg = os.path.join(path, f"b4_raw/{p}/b4_reg_seg.nii.gz")
            temp_list.append(seg)
    gt_list = temp_list

    temp_list = [gt_list, pred2d_list, pred3d_list]

    return temp_list

In [34]:
path = "/workspace/dwseon/dat/"
l = init_imgs(path)
gt_list = l[0]
pred2d_list = l[1]
pred3d_list = l[2]

In [35]:
print (f"Len check : {len(gt_list), len(pred2d_list), len(pred3d_list)}")

Len check : (19, 0, 19)


In [49]:
threeD_dice = {}
twoD_dice = {}
ensemble_dice = {}
prob2d, prob3d = 0, 0
ensem_out = os.path.join(path, "ensemble")
if not os.path.isdir(ensem_out):
    os.makedirs(ensem_out)
for i, gt in enumerate(gt_list):
    gt_img = nib.load(gt).get_fdata().astype(int)
    gt_img = torch.from_numpy(gt_img)
    for twoD in pred2d_list:
        if os.path.basename(gt.split("/b4_reg_seg.nii.gz")[0]) == os.path.basename(twoD).split(".nii.gz")[0]:
            # LOAD IMAGE
            pred2d_img = nib.load(twoD).get_fdata().astype(int)
            pred2d_img = torch.from_numpy(pred2d_img)
            base = os.path.basename(twoD).split(".nii.gz")[0]
            # DICE
            twoD_dice[base] = eval(pred2d_img, gt_img, smooth=0.00001)
            # SOFTMAX
            # prob2d = np.load(os.path.join(path, f"pred2d/{base}.npz"), allow_pickle=True)["softmax"] # this has 2 channels tho
    for threeD in pred3d_list:
        if os.path.basename(gt.split("/b4_reg_seg.nii.gz")[0]) == os.path.basename(threeD).split(".nii.gz")[0]:
            # LOAD IMAGE
            
            pred3d_img = nib.load(threeD).get_fdata().astype(int)
            pred3d_img = torch.from_numpy(pred3d_img)
            base = os.path.basename(threeD).split(".nii.gz")[0]
            # DICE
            threeD_dice[base] = eval(pred3d_img, gt_img, smooth=0.00001)
            # SORFTMAX
            # prob3d = np.load(os.path.join(path, f"pred3d/{base}.npz"), allow_pickle=True)["softmax"] # so let's use them separately
    # ENSEMBLE
    # ens_res = ensemble(prob2d, prob3d) # assuming [0] is pred and [1] is gt
    # ens_nii = nib.Nifti1Image(ens_res.reshape(prob2d[1].transpose(1,2,0).shape), affine=np.eye(4))
    # nib.save(ens_nii, os.path.join(ensem_out, f"{base}.nii.gz"))
    # ensemble_dice[base] = eval(torch.from_numpy(ens_res.astype(int)), gt_img, smooth=0.00001)

In [39]:
def avg_dice(dic):
    tot = 0
    for thing in dic:
        tot += dic[thing]
    if len(dic) != 0:
        tot = tot / len(dic)
    else:
        tot = "This plan contains 0 data."
    return tot

In [51]:
print(f"Average of \n- 2d dice: {avg_dice(twoD_dice)}\n- 3d dice: {avg_dice(threeD_dice)}\n- Ensemble: {avg_dice(ensemble_dice)}")

Average of 
- 2d dice: This plan contains 0 data.
- 3d dice: 0.8243799209594727
- Ensemble: This plan contains 0 data.


---
BELOW ARE TEST CELLS
---

In [160]:
img = nib.load(os.path.join(path, "pred2d/087.nii.gz")).get_fdata().astype(int)

In [102]:
a = torch.from_numpy(img)

In [103]:
a.unique()

tensor([0])

In [85]:
b = a.flatten()
b.shape

torch.Size([39845888])

In [192]:
aa = np.load("/workspace/dwseon/dat/pred2d/087.pkl", allow_pickle=True)

In [191]:
aa = aa["softmax"][0].transpose(1,2,0)
aa.shape

(1024, 1024, 38)

In [164]:
# shape don't matter but the len
data = aa['softmax']

In [165]:
data.shape

(2, 38, 1024, 1024)